#### Guiding Questions:
Who are the winners of the D1 division in the Germany Football Association (Bundesliga) in the last decade?

Which teams have been relegated in the past 10 years?

Does Octoberfest affect the performance of Bundesliga?

Which season of Bundesliga was the most competitive in the last decade?

What's the best month to watch Bundesliga?

In [1]:
import findspark
findspark.init()
import pyspark


from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [20]:
from pyspark.sql import Row
from pyspark.sql.functions import instr,col,column, expr, coalesce, broadcast
from pyspark.sql.functions import count, first, last,min, max, sum, avg, sumDistinct,sum_distinct, pow, desc, countDistinct, approx_count_distinct, dense_rank, rank
from pyspark.sql.functions import corr, round, bround, monotonically_increasing_id, mean, format_number, var_pop, stddev_pop,var_samp, stddev_samp, skewness, kurtosis, covar_pop, covar_samp
from pyspark.sql.functions import initcap , lower, upper, lit, ltrim, rtrim, rpad, lpad, trim, regexp_replace, regexp_extract, translate, collect_set, collect_list
from pyspark.sql.functions import when,current_date, current_timestamp, date_add, date_sub, datediff, to_date, months_between, dayofmonth,hour,dayofyear,month,year,weekofyear,date_format
from pyspark.sql.window import Window

In [67]:
df_matches  = spark.read.csv('data/Matches.csv',inferSchema=True,header=True)
df_teams  = spark.read.csv('data/Teams.csv',inferSchema=True,header=True)
df_team_matches  = spark.read.csv('data/Teams_in_Matches.csv',inferSchema=True,header=True)
df_team_id  = spark.read.csv('data/Unique_Teams.csv',inferSchema=True,header=True)

In [169]:
df_matches1=df_matches.filter((col('Season')>2000) & (col('Season')<=2010)).filter(col('Div')=='D1')
df_matches2=df_matches1.withColumn('HTP',when(col('FTR')=='H',3).when(col('FTR')=='D',1).otherwise(0))\
                                            .withColumn('ATP',when(col('FTR')=='A',3).when(col('FTR')=='D',1).otherwise(0))

In [170]:
df_matchesH3=df_matches2.groupby('Season', 'HomeTeam' ).agg(sum('FTHG').alias('GoalScored'),sum('FTAG').alias('GoalReceived'),\
                                                          sum('HTP').alias('TotalPoint')).withColumnRenamed('HomeTeam' ,'Team')
df_matchesA3=df_matches2.groupby('Season', 'AwayTeam').agg(sum('FTAG').alias('GoalScored'),sum('FTHG').alias('GoalReceived'),\
                                                          sum('ATP').alias('TotalPoint')).withColumnRenamed('AwayTeam' ,'Team')

In [171]:
df_matches4=df_matchesH3.union(df_matchesA3)
df_matches5=df_matches4.groupby('Season','Team').agg(sum('GoalScored').alias('GoalScored'),\
                                                    sum('GoalReceived').alias('GoalReceived'),\
                                                    sum('TotalPoint').alias('TotalPoint'))
#df_matches6=df_matches5.groupby('Season').max('TotalPoint')
df_matches6=df_matches5.groupby('Season').agg(max('TotalPoint').alias('MaxTotalPoint'))

In [172]:
#df_matches5.join(df_matches6,['Season'] ).show()
df_matches7=df_matches5.join(df_matches6,(df_matches5.Season == df_matches6.Season)&(df_matches5.TotalPoint == df_matches6.MaxTotalPoint))\
                        .drop(df_matches6.Season).drop(df_matches6.MaxTotalPoint).orderBy('Season',  ascending=True)

In [174]:
df_matches7.show()

+-------------+----------+------------+----------+------+
|         Team|GoalScored|GoalReceived|TotalPoint|Season|
+-------------+----------+------------+----------+------+
|     Dortmund|        62|          33|        70|  2001|
|Bayern Munich|        70|          25|        75|  2002|
|Werder Bremen|        79|          38|        74|  2003|
|Bayern Munich|        75|          33|        77|  2004|
|Bayern Munich|        67|          32|        75|  2005|
|    Stuttgart|        61|          37|        70|  2006|
|Bayern Munich|        68|          21|        76|  2007|
|    Wolfsburg|        80|          41|        69|  2008|
|Bayern Munich|        72|          31|        70|  2009|
|     Dortmund|        67|          22|        75|  2010|
+-------------+----------+------------+----------+------+

